In [1]:
import numpy as np
from meshplot import plot, subplot, interact
from scipy.stats import norm
import open3d as o3d
%run helpers_alg.ipynb
%run helpers_preproc.ipynb

In [2]:
mesh_dir = 'SHREC11/'
#ts = np.array([0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4])
ts = np.random.random(100) * 0.01

In [3]:
adjMats = []

for i in range(600):
    fName = 'T' + str(i) + '.obj'
    adj = obj_2_adj(mesh_dir + fName)
    adjMats.append(adj)

In [4]:
lapMats = adj_mat_to_lap_mat(adjMats)

In [ ]:
evals,Us = lap_mat_to_eigen(lapMats)
hkss = batch_heat_kernel_signature(ts, evals, Us)

In [ ]:
disSimMat = igl.all_pairs_distances(hkss,hkss,True)

In [ ]:
disSimMat = disSimMat / np.max(disSimMat)

In [ ]:
label = readLbl(mesh_dir+'labels.txt')
compResult = compareGroups(0,1,label,disSimMat)
np.mean(compResult)

In [ ]:
disSimMat2 = disSimMat + np.diag(np.ones(600))
print(divmod(np.argmin(disSimMat2),600))
print(np.min(disSimMat2))
print(label[59])
print(label[191])

In [ ]:
mat = np.empty([30,30])
compSelf = []
compOthers = []
for i in range(30):
    for j in range(0,i+1):
        compResult = compareGroups(i,j,label,disSimMat)
        mat[i,j] = compResult
        mat[j,i] = compResult
        if (i == j):
            compSelf.append(compResult)
        else:
            compOthers.append(compResult)

In [ ]:
print(np.min(compSelf))
print(compOthers.index(np.min(compOthers)))

In [ ]:
plt.matshow(mat)
self_mean = np.mean(compSelf)
self_std = np.std(compSelf)
others_mean = np.mean(compOthers)
others_std = np.std(compOthers)

In [ ]:
# Plot between -10 and 10 with .001 steps.
x_axis = np.arange(-0.4, 0.4, 0.001)

#plt.hist(compSelf)


plt.plot(x_axis, norm.pdf(x_axis, self_mean, self_std))
plt.plot(x_axis, norm.pdf(x_axis, others_mean, others_std))
plt.show()

In [ ]:
plt.hist(compOthers)

In [ ]:
print(self_mean,self_std)
print(others_mean,others_std)

In [ ]:

mesh = o3d.io.read_triangle_mesh(mesh_dir + "T0.obj")


In [ ]:
mesh

In [ ]:
sim_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(mesh, 250)

In [ ]:
sim_mesh
#sim_mesh = mesh

In [ ]:
sim_mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([sim_mesh])
